### 对因子做风格中性化

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from tqdm.notebook import tqdm
tqdm.pandas()

/mnt/datadisk2/aglv/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [21]:
facot_path = '/mnt/datadisk2/aglv/aglv/aglv_factor/bs_book/buy_l_s_ratio_r20_std_ratio.csv'
factor = pd.read_csv(facot_path)
factor = factor.set_index('date')
factor.index = [int(i) for i in factor.index]

In [2]:
cd /mnt/datadisk2/aglv/aglv/lab_aglv/forintern/

/mnt/datadisk2/aglv/aglv/lab_aglv/forintern


In [3]:
from DataDaily import DataDaily
datadaily = DataDaily()

load ./data_daily.h5...


In [23]:
clo_day = datadaily.adjclose
ret20 = clo_day / clo_day.shift(20) - 1
vol = datadaily.volume
vol20 = vol / vol.rolling(20).mean()
ret20 = ret20.loc[factor.index, factor.columns]
vol20 = vol20.loc[factor.index, factor.columns]

In [13]:
vol20

InstrumentID,BJ430017,BJ430047,BJ430090,BJ430139,BJ430198,BJ430300,BJ430418,BJ430425,BJ430476,BJ430478,...,SZ301589,SZ301591,SZ301596,SZ301600,SZ301603,SZ301606,SZ301607,SZ301608,SZ301611,SZ301618
date,,,,,,,,,,,,,,,,,,,,,
20171009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20171010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20171011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20171012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20171013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20241014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.152909,1.030012,1.088837,NaN,1.373072,1.229088,1.095867,1.198642,3.206596,NaN
20241015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.139331,1.226109,1.071101,NaN,1.458678,1.228873,1.215219,1.200943,2.252713,NaN
20241016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.818467,0.793463,2.042030,NaN,1.190078,0.847626,0.684256,0.809629,1.483118,NaN


In [17]:
factor.iloc[:, 2].name

'BJ430047'

In [8]:
# datadaily.mkt_cap

In [31]:
def neutralize_stock(factor_stock):
    factor_stock = factor_stock.dropna()
    stock_name = factor_stock.name
    X = pd.DataFrame({'ret20':ret20.loc[factor_stock.index, stock_name], 'vol20':vol20.loc[factor_stock.index, stock_name]})
    X = X.dropna()
    X = sm.add_constant(X)
    model = sm.OLS(factor_stock, X)
    res = model.fit()
    return res.resid

In [32]:
neu_factor = factor.progress_apply(neutralize_stock)
neu_factor

ValueError: zero-size array to reduction operation maximum which has no identity